## Preprocessing

In [1]:
# Import dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

#  Import and read the data.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial ID columns ('EIN' and 'NAME').
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other".
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts = app_type_counts.reset_index()
app_type_counts

,APPLICATION_TYPE,count
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


In [5]:
# Choose a cutoff value and create a list of application types to be replaced.
application_types_to_replace = []
app_type_counts.columns = ["APPLICATION_TYPE", "Count"]
for index, row in app_type_counts.iterrows():
  if row["Count"] < 528:
    application_types_to_replace.append(row["APPLICATION_TYPE"])

# Replace in dataframe.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check replacement.
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other".
class_type_counts = application_df["CLASSIFICATION"].value_counts()
class_type_counts = class_type_counts.reset_index()
class_type_counts.columns = ["CLASSIFICATION", "Count"]

# Filter by CLASSIFICATION value counts > 1
filtered_class_type_counts = class_type_counts[class_type_counts["Count"] > 1]
filtered_class_type_counts

,CLASSIFICATION,Count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [7]:
# Choose a cutoff value and create a list of classifications to be replaced.
classifications_to_replace = []
for index, row in class_type_counts.iterrows():
  if row["Count"] < 1883:
    classifications_to_replace.append(row["CLASSIFICATION"])

# Replace in dataframe.
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check replacement.
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# Convert categorical data to numeric.
categorical_cols = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"]
dummies = pd.get_dummies(application_df[categorical_cols], dtype=int)
application_df = pd.concat([application_df, dummies], axis=1)

application_df = application_df.drop(columns=categorical_cols, axis=1)

application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [9]:
# Split the preprocessed data into features and target arrays.
feature_cols = application_df.drop(columns=["IS_SUCCESSFUL"]).columns
X = application_df[feature_cols].values
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [10]:
# Create a StandardScaler instance.
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compiling, Training and Evaluating the Model

In [11]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [12]:
# Create a neural network model for binary classification.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/Applications/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model.
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model.
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 536us/step - accuracy: 0.6977 - loss: 0.5930
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - accuracy: 0.7246 - loss: 0.5580
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.7275 - loss: 0.5550
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 0.7338 - loss: 0.5488
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.7266 - loss: 0.5528
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step - accuracy: 0.7360 - loss: 0.5442
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.7328 - loss: 0.5466
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - accuracy: 0.7360 - loss: 0.5451
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.7279 - loss: 0.5509
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.7351 - loss: 0.5420
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.7282 - loss: 0.5498
Epoch 12/100
804/80

In [15]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 629us/step - accuracy: 0.7304 - loss: 0.5646
Loss: 0.5646385550498962, Accuracy: 0.7303789854049683


In [16]:
# Evaluate the model using a confusion matrix.
predictions = nn.predict(X_test_scaled)
y_pred = (predictions > 0.7).astype("int32")
confusion_matrix(y_test, y_pred)

268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step


array([[3301,  709],
       [2151, 2414]])

In [17]:
# Export our model to HDF5 file.
nn.save("deep-learning-model.h5")